## 3.1 决策树的构造

### 3.1.1 信息增益

In [1]:
# 3-1 计算给定数据集的香农熵
from math import log

# 计算香农熵
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        labelCounts[currentLabel] = labelCounts.get(currentLabel, 0) + 1
    
    shannonEnt = 0
    for key, value in labelCounts.items():
        prob = value / numEntries
        shannonEnt -= prob * log(prob, 2)
    return shannonEnt

In [102]:
# 生成样本数据集
def createDataSet():
    dataSet = [
        [1, 1, 'yes'],
        [1, 1, 'yes'],
        [1, 0, 'no' ],
        [0, 1, 'no' ],
        [0, 1, 'no' ]
    ]
    labels = ['不浮出水面是否可以生存', '是否有脚蹼']
    return dataSet, labels

# 增加maybe分类
def createDataSet2():
    dataSet = [
        [1, 1, 'maybe'],
        [1, 1, 'yes'],
        [1, 0, 'no' ],
        [0, 1, 'no' ],
        [0, 1, 'no' ]
    ]
    labels = ['不浮出水面是否可以生存', '是否有脚蹼']
    return dataSet, labels

In [64]:
dataSet, label = createDataSet()
dataSet2, label2 = createDataSet2()
print("两分类香农熵：{}".format(calcShannonEnt(dataSet)))
print("三分类香农熵：{}".format(calcShannonEnt(dataSet2)))

两分类香农熵：0.9709505944546686
三分类香农熵：1.3709505944546687


### 3.1.2 划分数据集

In [13]:
# 3-2 按照给定特征划分数据集
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [59]:
print(splitDataSet(dataSet, 0, 1))
print(splitDataSet(dataSet, 0, 0))

[[1, 'yes'], [1, 'yes'], [0, 'no']]
[[1, 'no'], [1, 'no']]


In [23]:
# 3-3 选择最好的数据集划分方式
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0
    bestFeatures = -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)  # 创建唯一的分类标签列表
        newEntropy = 0
        for value in uniqueVals:    # 计算每种划分方式的信息熵
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet) / len(dataSet)
            newEntropy += prob * calcShannonEnt(subDataSet)  
        infoGain = baseEntropy - newEntropy
        if (infoGain > bestInfoGain):   # 判断最好的信息增益
            bestInfoGain = infoGain
            bestFeatures = i
    return bestFeatures


In [27]:
print("两分类最优划分列：{}".format(chooseBestFeatureToSplit(dataSet)))
print("三分类最优划分列：{}".format(chooseBestFeatureToSplit(dataSet2)))

两分类最优划分列：0
三分类最优划分列：0


### 3.1.3 递归构建决策树

In [38]:
# 多数表决确定叶子节点
import operator

def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        classCount[vote] = classCount.get(vote, 0) + 1
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

In [119]:
# 3-4 创建树的函数代码
from enum import unique


def createTree(dataSet, labels):
    labels = labels[:]
    classList = [example[-1] for example in dataSet]
    # 边界条件1：遍历完所有可划分数据集的属性
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    # 边界条件2： 分支下实例都是相同分类
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    bestFeatures = chooseBestFeatureToSplit(dataSet)
    bestFeatLable = labels[bestFeatures]
    tree = {bestFeatLable: {}}
    del(labels[bestFeatures])    # 去除当前节点标签以构建子列表
    featValues = [example[bestFeatures] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subDataSet = splitDataSet(dataSet, bestFeatures, value)
        subFeatLabel = labels[:]
        tree[bestFeatLable][value] = createTree(subDataSet, subFeatLabel)
    return tree  
    

In [129]:
tree = createTree(dataSet, label)
tree2 = createTree(dataSet2, label2)
print("二分类树：{}".format(tree))
print("三分类树：{}".format(tree2))

二分类树：{'不浮出水面是否可以生存': {0: 'no', 1: {'是否有脚蹼': {0: 'no', 1: 'yes'}}}}
三分类树：{'属于鱼类': {0: 'no', 1: {'不属于鱼类': {0: 'no', 1: 'maybe'}}}}
